In [32]:
import pandas as pd
import os
import glob
import http.client
import json
import datetime

## Données des équipes de chaque ligue principales

In [33]:
connection = http.client.HTTPConnection('api.football-data.org')
headers = { 'X-Auth-Token': '52209beef1bc4a4e8b08fbf89163915f' }

connection.request('GET', 'https://api.football-data.org/v2/competitions/FL1/teams', None, headers )
equipes_fr = json.loads(connection.getresponse().read().decode())
connection.request('GET', 'https://api.football-data.org/v2/competitions/CL/teams', None, headers )
equipes_cl = json.loads(connection.getresponse().read().decode())
connection.request('GET', 'https://api.football-data.org/v2/competitions/PL/teams', None, headers )
equipes_eng = json.loads(connection.getresponse().read().decode())
connection.request('GET', 'https://api.football-data.org/v2/competitions/PD/teams', None, headers )
equipes_esp = json.loads(connection.getresponse().read().decode())
connection.request('GET', 'https://api.football-data.org/v2/competitions/SA/teams', None, headers )
equipes_ita = json.loads(connection.getresponse().read().decode())

### Fusion dans un dataframe

In [34]:
df_fr=pd.DataFrame(equipes_fr['teams'])
df_cl=pd.DataFrame(equipes_cl['teams'])
df_eng=pd.DataFrame(equipes_eng['teams'])
df_esp=pd.DataFrame(equipes_esp['teams'])
df_ita=pd.DataFrame(equipes_ita['teams'])

columns=['shortName','name']
fr_names=df_fr[columns]
cl_names=df_cl[columns]
eng_names=df_eng[columns]
esp_names=df_esp[columns]
ita_names=df_ita[columns]

df_teams=pd.DataFrame(columns=['shortName','name'])
df_teams=df_teams.append(fr_names,ignore_index=True)
df_teams=df_teams.append(eng_names,ignore_index=True)
df_teams=df_teams.append(esp_names,ignore_index=True)
df_teams=df_teams.append(ita_names,ignore_index=True)
df_teams=df_teams.append(cl_names,ignore_index=True)
df_teams=df_teams.drop_duplicates()
df_teams

,shortName,name
0,Toulouse,Toulouse FC
1,Brest,Stade Brestois 29
2,Marseille,Olympique de Marseille
3,Montpellier,Montpellier HSC
4,Lille,Lille OSC
...,...,...
154,Feronikeli,KF Feronikeli
155,Tre Penne,SP Tre Penne
156,Ararat-Armenia,FC Ararat-Armenia
157,Saburtalo,FC Saburtalo


## Puissance des équipes

In [35]:
puiss2019=pd.read_csv("Points_Equipe_Hist/Points_Equipes2019.csv",
                         sep=";",index_col=0)
puiss2018=pd.read_csv("Points_Equipe_Hist/Points_Equipes2018.csv",
                         sep=";",index_col=0)
puiss2017=pd.read_csv("Points_Equipe_Hist/Points_Equipes2017.csv",
                         sep=";",index_col=0)
puiss2016=pd.read_csv("Points_Equipe_Hist/Points_Equipes2016.csv",
                         sep=";",index_col=0)
puiss2015=pd.read_csv("Points_Equipe_Hist/Points_Equipes2015.csv",
                         sep=";",index_col=0)

FileNotFoundError: [Errno 2] File b'Points_Equipe_Hist/Points_Equipes2019.csv' does not exist: b'Points_Equipe_Hist/Points_Equipes2019.csv'

In [36]:
puiss2019

,Equipe,Points
0,Barcelone,15917
1,Liverpool,15510
2,Valence,12990
3,Flamengo,12536
4,ManchesterCity,12324
...,...,...
993,CardiffMU,74
994,NyasaBigBullets,69
995,Sarajevo,63
996,KR,59


### Fonctions de recherches et standardisation des noms

In [37]:
def most_frequent(List): 
    return max(set(List), key = List.count) 

In [38]:
def exception(equipe):
    
    # Espagne
    if (equipe=="Barça"):
        equipe="Barcelone"
        return equipe
    if (equipe=="Barcelona"):
        equipe="Barcelone"
        return equipe
    if (equipe=="Real Betis"):
        equipe="Bétis"
        return equipe
    if (equipe=="Valencia"):
        equipe="Valence"
        return equipe
    if (equipe=="Real Sociedad"):
        equipe="Sociedad"
        return equipe
    if (equipe=="Sevilla FC"):
        equipe="Séville"
        return equipe
    if (equipe=="Granada"):
        equipe="Grenade"
        return equipe
    if (equipe=="Mallorca"):
        equipe="Majorque"
        return equipe
    
    
    # Italie
    if (equipe=="Roma"):
        equipe="Rome"
        return equipe
    if (equipe=="Atleti"):
        equipe="Atlético"
        return equipe

    # France

    if (equipe=="Paris SG"):
        equipe="PSG"
        return equipe
    if (equipe=="St Etienne"):
        equipe="Saint-Étienne"
        return equipe
    if (equipe=="Rennes"):
        equipe="Stade Rennais"
        return equipe
    if (equipe=="Paris SG"):
        equipe="PSG"
        return equipe
    
    return equipe

In [39]:
def recherche_name(nom,data):
    v_mots=[]
    v=[]
    liste=[]
    mots = nom.split(' ')
    for mot in mots:
    # On évite les mots courts comme "de"
        if(len(str(mot))>3):
            v_mots.append(mot)
            # Récupération des équipes possèdant ces mots
            for v_mot in v_mots:
                x=data[data["Equipe"].str.contains(v_mot) == True]
                v.append(x["Equipe"].values)
            
    for item in v:
        if(len(item)>0):
            liste.append(item[0])
            
    if(len(liste)>0):
        equipe_trouvee=most_frequent(liste)
        good_name_str=equipe_trouvee
        
    else:
        # Cette équipe n'existe pas dans nos données donc surement trop faible
        good_name_str= None
    
    return good_name_str


def name_generic(equipe,df):
    name=None
    equipe=equipe.replace("Stade","")
    equipe=exception(equipe)
    x=df.loc[df['Equipe']==equipe]
    
    if(x.empty):
        name=recherche_name(equipe,df)
        #print("result :",name)
        return name
    
    else:    
        n=x['Equipe'].values
        name=n[0]
        return name


In [40]:
df_teams2019=df_teams.copy()
df_teams2018=df_teams.copy()
df_teams2017=df_teams.copy()
df_teams2016=df_teams.copy()
df_teams2015=df_teams.copy()

## Mise en commun des noms d'équipes de l'API et des csv puissance

In [41]:
df_teams2019['name_web']=df_teams2019['shortName'].apply(lambda x: name_generic(x,puiss2019))
df_teams2018['name_web']=df_teams2018['shortName'].apply(lambda x: name_generic(x,puiss2018))
df_teams2017['name_web']=df_teams2017['shortName'].apply(lambda x: name_generic(x,puiss2017))
df_teams2016['name_web']=df_teams2016['shortName'].apply(lambda x: name_generic(x,puiss2016))
df_teams2015['name_web']=df_teams2015['shortName'].apply(lambda x: name_generic(x,puiss2015))

## Récupération de la puissance

In [42]:
def recup_puissance(equipe,df):
    result=df["Points"].loc[df["Equipe"]==str(equipe)]
    if (result.empty):
        return 100  
    else: 
        x=result.values
        if (len(x)>1):
            x=x[0]
        return int(x)

In [43]:
def recup_puissance2(equipe,df):
    result=df["puissance"].loc[df["Equipe"]==str(equipe)]
    if (result.empty):
        return 100  
    else: 
        x=result.values
        if (len(x)>1):
            x=x[0]
        return int(x)

In [44]:
df_teams2019['puissance']=df_teams2019['name_web'].apply(lambda x: recup_puissance(x,puiss2019))
df_teams2018['puissance']=df_teams2018['name_web'].apply(lambda x: recup_puissance(x,puiss2018))
df_teams2017['puissance']=df_teams2017['name_web'].apply(lambda x: recup_puissance(x,puiss2017))
df_teams2016['puissance']=df_teams2016['name_web'].apply(lambda x: recup_puissance(x,puiss2016))
df_teams2015['puissance']=df_teams2015['name_web'].apply(lambda x: recup_puissance(x,puiss2015))

## Récupération des données historiques

In [45]:
year = 2014
while year <2019:
    
    if (year==2014):
        df_teams=df_teams2015
    if (year==2015):
        df_teams=df_teams2016
    if (year==2016):
        df_teams=df_teams2017
    if (year==2017):
        df_teams=df_teams2018
    if (year==2018):
        df_teams=df_teams2019
        
        
    ## Le dossier représente l'année du début de saison, donc - 1 an
    os.chdir("C:\\Users\\Amrta\\Documents\\GitHub\\projetannuel4IABD2\\ETAPE_3\\Scripts_Recup_Data\\Projet_Annuel_Data\\"+str(year))
    extension = 'csv'
    all_filenames = [i for i in glob.glob('*.{}'.format(extension))]
    # Fusion des fichiers csv dans le dossier
    csv2019 = pd.concat([pd.read_csv(f) for f in all_filenames ])
    
    data2019=csv2019[['Date','HomeTeam','AwayTeam','FTR','HTHG','HTAG','FTHG','FTAG']]
    data2019=data2019.dropna()
    
    
    ## Mise en commun des noms d'équipes de l'API et des csv historiques
    
    df_teams=df_teams.rename(columns={"shortName":"Equipe"})
    data2019['NameHome']=data2019['HomeTeam'].apply(lambda x: name_generic(x,df_teams))
    data2019['NameAway']=data2019['AwayTeam'].apply(lambda x: name_generic(x,df_teams))
    
    # Ajout des puissances relatifs à l'année
    data2019['puissanceHomeTeam']=data2019['NameHome'].apply(lambda x: recup_puissance2(x,df_teams))
    data2019['puissanceAwayTeam']=data2019['NameAway'].apply(lambda x: recup_puissance2(x,df_teams))
    
    os.chdir("C:\\Users\\Amrta\\Documents\\GitHub\\projetannuel4IABD2\\ETAPE_3\\Scripts_Recup_Data\\Projet_Annuel_Data\\All_years")
    data2019.to_csv(r"Matchs_Historique"+str(year)+".csv", sep=";", encoding='utf-8',index='False')
    
    year=year+1

## Fusion des données

In [46]:
all_filenames = [i for i in glob.glob('*.{}'.format(extension))]
# Fusion des fichiers csv dans le dossier
data_hist = pd.concat([pd.read_csv(f,sep=';',index_col=0) for f in all_filenames ])
data_hist

,Date,HomeTeam,AwayTeam,FTR,HTHG,HTAG,FTHG,FTAG,NameHome,NameAway,puissanceHomeTeam,puissanceAwayTeam
0,08/08/14,Reims,Paris SG,D,2.0,1.0,2.0,2.0,Stade de Reims,PSG,2021,9482
1,09/08/14,Bastia,Marseille,D,1.0,2.0,3.0,3.0,NaN,Marseille,100,4683
2,09/08/14,Evian Thonon Gaillard,Caen,A,0.0,3.0,0.0,3.0,NaN,NaN,100,100
3,09/08/14,Guingamp,St Etienne,A,0.0,1.0,0.0,2.0,NaN,Saint-Étienne,100,5328
4,09/08/14,Lille,Metz,D,0.0,0.0,0.0,0.0,Lille,FC Metz,3768,723
...,...,...,...,...,...,...,...,...,...,...,...,...
375,18/05/2019,Levante,Ath Madrid,D,2.0,0.0,2.0,2.0,Levante,Real Madrid,5198,11475
376,18/05/2019,Sevilla,Ath Bilbao,H,1.0,0.0,2.0,0.0,Sevilla FC,NaN,9335,100
377,18/05/2019,Valladolid,Valencia,A,0.0,1.0,0.0,2.0,Valladolid,NaN,4109,100
378,19/05/2019,Eibar,Barcelona,D,2.0,2.0,2.0,2.0,Eibar,NaN,4455,100


### Dans le script de l'API les dates sont au format YYYY-MM-DD, il faut donc adapter les dates des données

De plus, il existe deux formats de dates différents dans ces données

In [47]:
def generic_date(date):
    
    if (len(date)>8):
        annee=date[6:10]
        mois=date[3:5]
        jour=date[0:2]
    else:
        annee="20"+date[6:8]
        mois=date[3:5]
        jour=date[0:2]
    
    result=datetime.datetime(int(annee),int(mois),int(jour))
    return result

In [48]:
data_hist['date']=data_hist['Date'].apply(lambda x: generic_date(x))

## Adaptation au bon format des scores

In [49]:
data_hist=data_hist.rename(columns={"HomeTeam":"homeTeam","AwayTeam":"awayTeam","FTR":"score",
                                    "HTHG":"goalHomeTeam_HalfTime","HTAG":"goalAwayTeam_HalfTime",
                                    "FTHG":"goalHomeTeam_FullTime","FTAG":"goalAwayTeam_FullTime"})

In [50]:
def generic_score(score):
    if score=="D":
        result=0
    if score=="H":
        result=1
    if score=="A":
        result=2
        
    return result

In [51]:
data_hist['score']=data_hist['score'].apply(lambda x: generic_score(x))

In [52]:
data_hist=data_hist[['date','homeTeam',"awayTeam","score","goalHomeTeam_HalfTime",
                    "goalAwayTeam_HalfTime","goalHomeTeam_FullTime","goalAwayTeam_FullTime",
                     "puissanceHomeTeam","puissanceAwayTeam"]]
data_hist

,date,homeTeam,awayTeam,score,goalHomeTeam_HalfTime,goalAwayTeam_HalfTime,goalHomeTeam_FullTime,goalAwayTeam_FullTime,puissanceHomeTeam,puissanceAwayTeam
0,2014-08-08,Reims,Paris SG,0,2.0,1.0,2.0,2.0,2021,9482
1,2014-08-09,Bastia,Marseille,0,1.0,2.0,3.0,3.0,100,4683
2,2014-08-09,Evian Thonon Gaillard,Caen,2,0.0,3.0,0.0,3.0,100,100
3,2014-08-09,Guingamp,St Etienne,2,0.0,1.0,0.0,2.0,100,5328
4,2014-08-09,Lille,Metz,0,0.0,0.0,0.0,0.0,3768,723
...,...,...,...,...,...,...,...,...,...,...
375,2019-05-18,Levante,Ath Madrid,0,2.0,0.0,2.0,2.0,5198,11475
376,2019-05-18,Sevilla,Ath Bilbao,1,1.0,0.0,2.0,0.0,9335,100
377,2019-05-18,Valladolid,Valencia,2,0.0,1.0,0.0,2.0,4109,100
378,2019-05-19,Eibar,Barcelona,0,2.0,2.0,2.0,2.0,4455,100


# Fonctions de générations des nouvelles données

In [53]:
def nb_matchs(df_verif,name):
    matchs=0
    for row in df_verif.itertuples(index=False):
        home=row.homeTeam
        away=row.awayTeam
        if (home==name):
            matchs+=1
        if (away==name):
            matchs+=1
    return matchs

def nb_goals_HalfTime(df_verif,name):
    goals_total=0
    for row in df_verif.itertuples(index=False):
        home=row.homeTeam
        away=row.awayTeam
        if (home==name):
            goals=row.goalHomeTeam_HalfTime
            goals_total+=goals
        if (away==name):
            goals=row.goalAwayTeam_HalfTime
            goals_total+=goals
        
    return goals_total


def nb_goals_FullTime(df_verif,name):
    goals_total=0
    for row in df_verif.itertuples(index=False):
        home=row.homeTeam
        away=row.awayTeam
        if (home==name):
            goals=row.goalHomeTeam_FullTime
            goals_total+=goals
        if (away==name):
            goals=row.goalAwayTeam_FullTime
            goals_total+=goals
        
    return goals_total

def nb_conceded_goals_HalfTime(df_verif,name):
    goals_total=0
    for row in df_verif.itertuples(index=False):
        home=row.homeTeam
        away=row.awayTeam
        if (home==name):
            goals=row.goalAwayTeam_HalfTime
            goals_total+=goals
        if (away==name):
            goals=row.goalHomeTeam_HalfTime
            goals_total+=goals
        
    return goals_total

def nb_conceded_goals_FullTime(df_verif,name):
    goals_total=0
    for row in df_verif.itertuples(index=False):
        home=row.homeTeam
        away=row.awayTeam
        if (home==name):
            goals=row.goalAwayTeam_FullTime
            goals_total+=goals
        if (away==name):
            goals=row.goalHomeTeam_FullTime
            goals_total+=goals
        
    return goals_total


def compte_win(df_verif,name):
    win_cons=0
    for row in df_verif.itertuples(index=False):
        home=row.homeTeam
        away=row.awayTeam
        score=row.score
        if ((home==name) and (score==1)):
            win_cons+=1
        if ((away==name) and (score==2)):
            win_cons+=1
        if ((home==name) and (score==2)):
            win_cons=0
        if (((away==name) and (score==1)) or (score==0)):
            win_cons=0
    return win_cons

def compte_lose(df_verif,name):
    lose_cons=0
    for row in df_verif.itertuples(index=False):
        home=row.homeTeam
        away=row.awayTeam
        score=row.score
        if ((home==name) and (score==2)):
            lose_cons+=1
        if ((away==name) and (score==1)):
            lose_cons+=1
        if ((home==name) and (score==1)):
            lose_cons=0
        if (((away==name) and (score==2)) or (score==0)):
            lose_cons=0
    return lose_cons


# Génération des données NbMatchs, Buts, Victoires, Défaites consécutives

In [54]:

def matchs_anterieurs(x,df):
    date=x['date']
    anterieur=df.loc[df['date']< date]    
    home=x["homeTeam"]
    away=x["awayTeam"]
    
    nb_win_home=0 
    nb_win_away=0 
    nb_lose_home=0 
    nb_lose_away=0 
    nb_matchs_home=0 
    nb_matchs_away=0
    nb_goals_home_Half=0 
    nb_goals_home_Full=0 
    nb_goals_away_Half=0 
    nb_goals_away_Full=0
    nb_goals_conceded_home_Half=0
    nb_goals_conceded_away_Half=0
    nb_goals_conceded_home_Full=0
    nb_goals_conceded_away_Full=0
    
    
    if(anterieur.empty): 
        return nb_matchs_home,nb_matchs_away, nb_win_home, nb_win_away, nb_lose_home, nb_lose_away,nb_goals_home_Half, nb_goals_home_Full, nb_goals_away_Half, nb_goals_away_Full, nb_goals_conceded_home_Half, nb_goals_conceded_away_Half, nb_goals_conceded_home_Full, nb_goals_conceded_away_Full
    else:
        verifHome=anterieur.loc[anterieur['homeTeam']==home]
        verifHome2=anterieur.loc[anterieur['awayTeam']==home]
        
        verifAway=anterieur.loc[anterieur['homeTeam']==away]
        verifAway2=anterieur.loc[anterieur['awayTeam']==away]
        
        df_verif=pd.concat([verifHome,verifHome2])
        df_verif=df_verif.sort_values(by = 'date')
        
        df_verif2=pd.concat([verifAway,verifAway2])
        df_verif2=df_verif2.sort_values(by = 'date')
        
        if (df_verif.empty):
            nb_win_home=0
        else:
            nb_win_home=compte_win(df_verif,home)
            nb_lose_home=compte_lose(df_verif,home)
            
            nb_matchs_home=nb_matchs(df_verif,home)
            nb_goals_home_Half=nb_goals_HalfTime(df_verif,home)
            nb_goals_home_Full=nb_goals_FullTime(df_verif,home)
            nb_goals_conceded_home_Half=nb_conceded_goals_HalfTime(df_verif,home)
            nb_goals_conceded_home_Full=nb_conceded_goals_FullTime(df_verif,home)
        
        if(df_verif2.empty):
            nb_win_away=0
        
        else:
            nb_win_away=compte_win(df_verif2,away)
            nb_lose_away=compte_lose(df_verif2,away)
            
            nb_matchs_away=nb_matchs(df_verif2,away)
            nb_goals_away_Half=nb_goals_HalfTime(df_verif2,away)
            nb_goals_away_Full=nb_goals_FullTime(df_verif2,away)
            nb_goals_conceded_away_Half=nb_conceded_goals_HalfTime(df_verif2,away)
            nb_goals_conceded_away_Full=nb_conceded_goals_FullTime(df_verif2,away)
            
        
    return nb_matchs_home,nb_matchs_away, nb_win_home, nb_win_away, nb_lose_home, nb_lose_away,nb_goals_home_Half, nb_goals_away_Half, nb_goals_home_Full, nb_goals_away_Full, nb_goals_conceded_home_Half, nb_goals_conceded_away_Half, nb_goals_conceded_home_Full, nb_goals_conceded_away_Full


In [55]:
## Script long
data_hist['nb_matchs_home'],data_hist['nb_matchs_away'],data_hist['win_cons_home'],data_hist['win_cons_away'],data_hist['lose_cons_home'], data_hist['lose_cons_away'],data_hist['nb_goals_home_Half'],data_hist['nb_goals_away_Half'],data_hist['nb_goals_home_Full'],data_hist['nb_goals_away_Full'],data_hist['nb_goals_conceded_home_Half'],data_hist['nb_goals_conceded_away_Half'],data_hist['nb_goals_conceded_home_Full'],data_hist['nb_goals_conceded_away_Full']=zip(*data_hist.apply
(lambda x: matchs_anterieurs(x,data_hist),axis=1))



## Moyennes

In [56]:
def moyenne(matchs,goals):
    moyenne=0
    if matchs > 0:
        moyenne=goals/matchs
    return moyenne

In [57]:
data_hist['moy_goals_home']=data_hist.apply(lambda x: moyenne(x['nb_matchs_home'],x['nb_goals_home_Full']), axis=1)
data_hist['moy_goals_away']=data_hist.apply(lambda x: moyenne(x['nb_matchs_away'],x['nb_goals_away_Full']), axis=1)
data_hist['moy_goals_half_home']=data_hist.apply(lambda x: moyenne(x['nb_matchs_home'],x['nb_goals_home_Half']), axis=1)
data_hist['moy_goals_half_away']=data_hist.apply(lambda x: moyenne(x['nb_matchs_away'],x['nb_goals_away_Half']), axis=1)
data_hist['moy_goals_conceded_home']=data_hist.apply(lambda x: moyenne(x['nb_matchs_away'],x['nb_goals_conceded_home_Full']), axis=1)
data_hist['moy_goals_conceded_away']=data_hist.apply(lambda x: moyenne(x['nb_matchs_away'],x['nb_goals_conceded_away_Full']), axis=1)
data_hist['moy_goals_conceded_half_home']=data_hist.apply(lambda x: moyenne(x['nb_matchs_away'],x['nb_goals_conceded_home_Half']), axis=1)
data_hist['moy_goals_conceded_half_away']=data_hist.apply(lambda x: moyenne(x['nb_matchs_away'],x['nb_goals_conceded_away_Half']), axis=1)


In [58]:
data_hist.sort_values(by = 'date')

,date,homeTeam,awayTeam,score,goalHomeTeam_HalfTime,goalAwayTeam_HalfTime,goalHomeTeam_FullTime,goalAwayTeam_FullTime,puissanceHomeTeam,puissanceAwayTeam,...,nb_goals_conceded_home_Full,nb_goals_conceded_away_Full,moy_goals_home,moy_goals_away,moy_goals_half_home,moy_goals_half_away,moy_goals_conceded_home,moy_goals_conceded_away,moy_goals_conceded_half_home,moy_goals_conceded_half_away
0,2014-08-08,Reims,Paris SG,0,2.0,1.0,2.0,2.0,2021,9482,...,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,2014-08-09,Bastia,Marseille,0,1.0,2.0,3.0,3.0,100,4683,...,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,2014-08-09,Evian Thonon Gaillard,Caen,2,0.0,3.0,0.0,3.0,100,100,...,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,2014-08-09,Guingamp,St Etienne,2,0.0,1.0,0.0,2.0,100,5328,...,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,2014-08-09,Lille,Metz,0,0.0,0.0,0.0,0.0,3768,723,...,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
373,2019-05-24,Dijon,Toulouse,1,0.0,1.0,2.0,1.0,2106,1627,...,248.0,310.0,1.165563,1.030837,0.596026,0.422907,1.092511,1.365639,0.418502,0.559471
372,2019-05-24,Caen,Bordeaux,2,0.0,1.0,0.0,1.0,100,2611,...,342.0,277.0,0.973568,1.273128,0.431718,0.572687,1.506608,1.220264,0.660793,0.506608
371,2019-05-24,Angers,St Etienne,0,0.0,0.0,1.0,1.0,2974,3866,...,236.0,241.0,1.084656,1.233480,0.470899,0.440529,1.039648,1.061674,0.480176,0.471366
370,2019-05-24,Amiens,Guingamp,1,1.0,0.0,2.0,1.0,2440,100,...,93.0,342.0,0.880000,1.123348,0.373333,0.409692,0.409692,1.506608,0.145374,0.550661


In [59]:
data_hist.to_csv(r"Data_Final/Data_Historical_Train.csv", sep=";", encoding='utf-8')